In [14]:
from trustcall import create_extractor
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

conversation = [HumanMessage(content="Hi, I'm Lance."), 
                AIMessage(content="Nice to meet you, Lance."), 
                HumanMessage(content="I really like biking around San Francisco.")]

# Schema
class UserProfile(BaseModel):
    """ User profile schema with typed fields"""
    user_name: str = Field(description="The user's preferred name")
    interests: List[str]

model = ChatOpenAI(model = "gpt-4o-mini")

# Create trustcall extractor
trustcall_extractor = create_extractor(
    model,
    tools = [UserProfile],
    tool_choice="UserProfile"
)

# Instruction
system_message = "Extract the user profile form the following converstion"

# Invoke the extractor
result = trustcall_extractor.invoke({"messages":[SystemMessage(content = system_message)]+ conversation})
result


{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hMX3rIRPApzkphpX7xwKXM9U', 'function': {'arguments': '{"user_name":"Lance","interests":["biking","San Francisco"]}', 'name': 'UserProfile'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 122, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-fa7ea9df-2391-4fb3-97ac-36a81f1a1422-0', tool_calls=[{'name': 'UserProfile', 'args': {'user_name': 'Lance', 'interests': ['biking', 'San Francisco']}, 'id': 'call_hMX3rIRPApzkphpX7xwKXM9U', 'type': 'tool_call'}], usage_metadata={'input_tokens': 122, 'output_tokens': 18, 'total_tokens': 140, 'inp

In [7]:
from pprint import pprint
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  UserProfile (call_rmX8XIN0q7j8YIcgLp1eMQXG)
 Call ID: call_rmX8XIN0q7j8YIcgLp1eMQXG
  Args:
    user_name: Lance
    interests: ['biking', 'San Francisco']


In [15]:
schema = result["responses"]
schema[0].model_dump()

{'user_name': 'Lance', 'interests': ['biking', 'San Francisco']}

In [12]:
# Update the conversation
updated_conversation = [HumanMessage(content="Hi, I'm Lance."), 
                        AIMessage(content="Nice to meet you, Lance."), 
                        HumanMessage(content="I really like biking around San Francisco."),
                        AIMessage(content="San Francisco is a great city! Where do you go after biking?"),
                        HumanMessage(content="I really like to go to a bakery after biking."),]

# Update the instructions
system_message = f""" Update the memory (JSON doc) to incorporate new information from the following conversation."""

# Invoke the extractor with the update instruction and existing profile with the corrsponding tool name (UserProfile)
result = trustcall_extractor.invoke({"messages": [SystemMessage(content=system_message)]+updated_conversation},{"exsiting": {"UserProfile": schema[0].model_dump()}})

In [13]:
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  UserProfile (call_zDAjJUE8ZIK2JsM6dpLkEYVT)
 Call ID: call_zDAjJUE8ZIK2JsM6dpLkEYVT
  Args:
    user_name: Lance
    interests: ['biking', 'baking', 'visiting bakeries']
